# YOLO V1
[You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/pdf/1506.02640.pdf)
## 贡献
Yolov1是第一个One-stage的目标检测算法，将Two-stage的分类和回归问题统一为了一个回归问题。

**定义**:

* **栅格(Grid)**
    Yolo把输入图片划分为 $S \times S$ 个栅格，每个栅格负责检测中心落在该格内的对象。原始论文中设为 $7 \times 7$ ，每个栅格预测 $B$ 个Bounding Box， $C$个条件类别概率。
    <div align =center><img src = "./img/栅格.jpg", style="height:20%;width:20%;"/></div>
* **边界框(Bounding Box)**
    原始论文中设为2，每个栅格的$B$个框都是预测同一类物体。


**核心思想**：将整张图片作为网络输入，直接在输出层对Bounding Box的位置和类别进行回归。

**实现方法**：将图片分为$S \times S$个栅格，如果某个Object的中心点落在了这个栅格中，就由这个栅格负责预测这个Object。每个栅格需要预测$B$个BoundingBox的位置信息$(x,y,h,w)$和置信度$(c)$，此外还要预测这个Object是每个类别的条件概率。

* **$(x,y,h,w)$**: 分别代表的是Bounding Box的中心位置和宽高

    $(x,y)$是预测的BBox中心相对于栅格的offset

    $(w,j)$是预测的BBox相对于整个图片的比例

* **$confidence$**: 代表了所预测的box中是否存在Object，以及该Box预测准确度。公式如下：
    $$ confidence = P_r(Object)\times IOU_{pred}^{truth}$$
    $P_r(Object)$表示Object是否落在这个grid里，如果grid里有Object,$P_r(Object) = 1$，否则，$P_r(Object) = 0$；
    
    $IOU$表示的是该BoundingBox的准确度,$pred$是预测出的BBox的区域位置，$truth$是标签区域位置，$IOU$就是求两个区域的交集比并集

<div align =center><img src = "./img/IOU.png", style="height:20%;width:20%;"/></div>

* **条件概率**：定义为$P_r(Class_i|Object)$，表示该栅格存在物体的情况下，属于第i类的概率。测试时每个单元格最终输出的概率定义为:

$$P_r(Class_i|Object) \times P_r(Object) \times IOU_{pred}^{truth} = P_r(Class_i) \times IOU_{pred}^{truth}$$

* **$ground truth$**: 模型标签为(x,y,w,h,cls)的五元组。置信度是由(x,y,w,h)计算出来的。概率对应cls独热编码。


## 算法步骤
### 1. 神经网络提取网格
<div align=center><img src = "./img/yolov1_network.png", style="height:80%;width:80%;"></img></div>

Yolov1原论文中借鉴GoogleNet网络结构，24个卷积层+2个全连接层。卷积层最后输出为$1024 \times 7 \times 7$，经过Flatten，再经过两层全连接，先映射为$4096$维，再映射并reshape为$30 \times 7 \times 7$。

Yolov1先用224*224在ImageNet进行预训练,用分类任务来训练特征提取器，然后再用$448 \times 448$进行目标检测的训练，这样就会导致训练过程既要学习如何去检测物体，又要去适应新的分辨率。

**损失函数**
<div align=center><img src = "./img/yolov1_loss.png", style="height:80%;width:80%;"></img></div>

$l_{i,j}^{obj}$ 标志当前网格是否有物体；
$\lambda_{coord}、\lambda_{noobj}$ 都是权重因子，超参数。

红框内：坐标误差损失。分别是预测的中心坐标与真实中心坐标的欧氏距离，预测宽高与真是宽高的欧氏距离。 

绿框内：置信度损失。分两种情况，当前网格是否有物体，如果有，直接计算置信度的均方误差，如果没有按第二行计算。

橙框内：预测概率误差。 当前网格p与真实p的误差。这里的p是 $P_r(Class_i|Object)$

In [1]:
# YOLO V1

### 2. NMS(非极大值抑制)
在第一部分中，每个栅格都会产出$B$个Bounding Box，并不是所有的Bounding Box都是有用的，因此需要去冗余，从堆叠的边框中挑出最好的那一个。
<div align=center><img src = "./img/NMS1.png", style="height:30%;width:30%;"></img></div>

NMS:一种局部最大搜索，局部代表的是一个邻域，邻域维度，大小都是可变参数，即在邻域中寻找一个局部最大值。

一维的NMS算法：判断数组I[W]的元素I[i]是否是局部极大值，邻域为左邻 i - 1 和右邻 i + 1
<div align=center><img src = "./img/NMS.png", style="height:30%;width:30%;"></img></div>

推广到二维：

算法输入：网络输出的所有候选框（位置信息及置信度）

算法输出：输入的自己，表示筛选之后的边界框

算法流程：
* 1.将所有的框按类别划分，并剔除背景类，因为无需NMS。
* 2.对每个物体类中的边界框(B_BOX)，按照分类置信度降序排列。
* 3.在某一类中，选择置信度最高的边界框B_BOX1，将B_BOX1从输入列表中去除，并加入输出列表。
* 4.逐个计算B_BOX1与其余B_BOX2的交并比IoU，若IoU(B_BOX1,B_BOX2) > 阈值TH，则在输入去除B_BOX2。
* 5.重复步骤3~4，直到输入列表为空，完成一个物体类的遍历。
* 6.重复2~5，直到所有物体类的NMS处理完成。
* 7.输出列表，算法结束

In [ ]:
# Yolov1预训练模型